# Estimation of gradient index n2 from measured focal range of GRIN-rod F

In [ ]:
import numpy as np
import sympy as sym
import math
import matplotlib.pyplot as plt

plt.rcParams.update({
    'font.size': 10,
    'legend.frameon': True
})

def plot_cross(ax, pt, bounds):
    ax.plot([np.min(bounds[0]), np.max(bounds[0])], [pt[1], pt[1]])
    ax.plot([pt[0], pt[0]], [np.min(bounds[1]), np.max(bounds[1])])

n0 = 1.73

# The formula is initialy taken from the Yariv's 'Quantun electronics' p.113.
def focus(n0, n2, L):
    return 1/n0/np.sqrt(n2/n0)/np.tan(np.sqrt(n2/n0)*L)

def focus_neg(n0, n2, L):
    return -1/n0/np.sqrt(-n2/n0)/np.tanh(np.sqrt(-n2/n0)*L)


## Direct calculation of F from known n2

In [ ]:
# Positive lens
n2 = np.linspace(start=0.1, stop=100, num=10)
fig, ax = plt.subplots(figsize=(6,4))
ax.set_xlabel('Gradient constant $n_2 \;\; (1/m^2)$')
ax.set_ylabel('Focus range, m')
ax.set_yscale('log')
ax.set_xscale('log')
ax.grid(True, linestyle=':')
for L in [0.001, 0.01, 0.1]:
    F = focus(n0, n2, L)
    ax.plot(n2, F, label=f'L={L}')
ax.legend(title='Rod length, m')

# Negative lens
n2 = np.linspace(start=-100, stop=-0.1, num=10)
fig, ax = plt.subplots(figsize=(6,4))
ax.set_xlabel('Gradient constant (negative) $n_2 \;\; (1/m^2)$')
ax.set_ylabel('Focus range (negative), m')
ax.set_yscale('log')
ax.set_xscale('log')
ax.grid(True, linestyle=':')
for L in [0.001, 0.01, 0.1]:
    F = focus_neg(n0, n2, L)
    # Have to inverse because want to plot in log scale
    ax.plot(-n2, -F, label=f'L={L}')
ax.legend(title='Rod length, m')

In [ ]:
# Just curious what error is when we approximate sin(x) = x and cos(x) = 1

n2 = np.linspace(start=0.1, stop=100, num=10)
fig, ax = plt.subplots(figsize=(6,4))
ax.set_xlabel('Gradient constant $n_2 \;\; (1/m^2)$')
ax.set_ylabel('Approximation error, %')
ax.grid(True, linestyle=':')
for L in [0.01, 0.05, 0.1]:
    F1 = focus(n0, n2, L)
    F2 = 1/L/n2
    ax.plot(n2, (F2-F1)/F1*100, label=f'L={L}')
ax.legend(title='Rod length, m')

## Inverse calculation of n2 from measured F 

In [ ]:
# Fix some values to keep them same between different calc approaches
L = 0.1
F = 1.45
F_neg = -F

# Reduced formulas with `x=sqrt(n2/n0)*L`
# The below `g` is stand for `gamma=sqrt(n2/n0)`

# Positive lens:
# F = 1 / (tan(g*L) * n0 * g)
# F * tan(g*L) * n0 * g = 1
# F * tan(g*L) * n0 * g*L = L
# F * tan(x) * n0 * x - L = 0
def n2_equat(x):
    return x*F*n0*np.tan(x) - L

# Negative lens:
# F = -1 / (tanh(g*L) * n0 * g)
# F * tanh(g*L) * n0 * g = -1
# F * tanh(g*L) * n0 * g*L = -L
# F * tanh(x) * n0 * x + L = 0
def n2_equat_neg(x):
    return x*F_neg*n0*np.tanh(x) + L


In [ ]:
# Analytical solution
# Use reduced formula with `x=sqrt(n2/n0)*L` and solve agains `x`

print('**** Positive lens:')
x = sym.Symbol('x')
x_roots = [sym.nsolve(x*F*n0*sym.sin(x) - L*sym.cos(x), x0) for x0 in [0, 0.1, 0.5, 1]]
print('Possible solutions for x:', x_roots)
x_root = np.float64(np.min([x for x in x_roots if x > 0]))
n2_root = np.float64(n0*(x_root/L)**2)
F_root = focus(n0, n2_root, L)
print('Solution: x_root =', x_root, 'n2 =', n2_root)
print(f'Back calculated F={F_root}, error={(F_root-F)/F*100}%')

print('\n**** Negative lens:')
x = sym.Symbol('x')
x_roots = [sym.nsolve(x*F_neg*n0*sym.sinh(x) + L*sym.cosh(x), x0) for x0 in [0, 0.1, 0.5, 1]]
print('Possible solutions for x:', x_roots)
x_root_neg = np.float64(np.min([x for x in x_roots if x > 0]))
n2_root_neg = np.float64(-n0*(x_root_neg/L)**2)
F_root = focus_neg(n0, n2_root_neg, L)
print('Solution: x_root =', x_root_neg, 'n2 =', n2_root_neg)
print(f'Back calculated F={F_root}, error={(F_root-F_neg)/F_neg*100}%')

In [ ]:
# Analytical solution
# Use the full formula and solve agains `n2`
# Just to compare with the previous solution
#
# F = 1 / (tan(sqrt(n2/n0)*L) * n0 * sqrt(n2/n0))
# F * tan(sqrt(n2/n0)*L) * n0 * sqrt(n2/n0) = 1
# F * sin(sqrt(n2/n0)*L) * n0 * sqrt(n2/n0) = cos(sqrt(n2/n0)*L)
# F * sin(sqrt(n2/n0)*L) * n0 * sqrt(n2/n0) - cos(sqrt(n2/n0)*L) = 0

print('**** Positive lens:')
x = sym.Symbol('x')
n2_roots = [sym.nsolve(F*sym.sin(sym.sqrt(x/n0)*L)*n0*sym.sqrt(x/n0) - sym.cos(sym.sqrt(x/n0)*L), x0) for x0 in [0.1, 0.5, 1]]
print('Possible solutions for n2:', n2_roots)
n2_root_1 = np.float64(np.min([n2 for n2 in n2_roots if n2 > 0]))
F_root = focus(n0, n2_root_1, L)
print('Solution: n2 =', n2_root_1)
print(f'Diff with prev solution: diff_n2={abs(n2_root_1-n2_root)}')
print(f'Back calculated F={F_root}, error={(F_root-F)/F*100}%')

print('\n**** Negative lens:')
x = sym.Symbol('x')
n2_roots = [sym.nsolve(sym.sqrt(n0/x)/n0*sym.cosh(sym.sqrt(x/n0)*L) + F_neg*sym.sinh(sym.sqrt(x/n0)*L), x0) for x0 in [0.1, 0.5, 1]]
print('Possible solutions for n2:', n2_roots)
n2_root_1_neg = -np.float64(np.min([n2 for n2 in n2_roots if n2 > 0]))
F_root = focus_neg(n0, n2_root_1_neg, L)
print('Solution: n2 =', n2_root_1_neg)
print(f'Diff with prev solution: diff_n2={abs(n2_root_1_neg-n2_root_neg)}')
print(f'Back calculated F={F_root}, error={(F_root-F_neg)/F_neg*100}%')


In [ ]:
# Plot the reduced formula against `x` to find out a shape of graph
# This shape affects what a manual solution looks like
print('**** Positive lens:')
x = np.linspace(start=0, stop=0.3, num=100)
y = n2_equat(x)
print(f'y_min ={np.min(y)}, y_max={np.max(y)}')
print('y @ x_root =', n2_equat(x_root))
fig, ax = plt.subplots(figsize=(6,4))
ax.grid(True, linestyle=':')
ax.set_xlabel('$x = \sqrt {\\frac {n_2} {n_0}} L $')
ax.set_ylabel('$ x \; F \; n_0 \; \\tan(x) - L $')
ax.plot(x, y, label='F > 0')
plot_cross(ax, pt=[x_root, 0], bounds=[x, y])
ax.legend()

print('\n**** Negative lens:')
y = n2_equat_neg(x)
print(f'y_min ={np.min(y)}, y_max={np.max(y)}')
print('y @ x_root =', n2_equat_neg(x_root_neg))
fig, ax = plt.subplots(figsize=(6,4))
ax.grid(True, linestyle=':')
ax.set_xlabel('$x = \sqrt {\\frac {|n_2|} {n_0}} L $')
ax.set_ylabel('$ x \; F \; n_0 \; \\tanh(x) + L $')
ax.plot(x, y, label='F < 0')
plot_cross(ax, pt=[x_root_neg, 0], bounds=[x, y])
ax.legend()


In [ ]:
# Calculate manually - Positive lens

x_next = 0
x_prev = 0
y_next = n2_equat(x_next)
iters1 = 0
while y_next < 0:
    iters1 += 1
    x_prev = x_next
    x_next += 0.1
    y_next = n2_equat(x_next)
#print(f'x_prev={x_prev}, x_next={x_next}, y_next={y_next}')
y_mid = y_next
iters2 = 0
while abs(y_mid) > 1e-8:
    iters2 += 1
    x_mid = (x_prev + x_next) / 2
    y_mid = n2_equat(x_mid)
    #print(f'x_mid={x_mid}, y_mid={y_mid}, y_next={y_next}')
    if y_next > 0 and y_mid < 0:
        x_prev = x_mid
    else:
        x_next = x_mid
        y_next = y_mid
n2 = n0*(x_mid/L)**2
F1 = focus(n0, n2, L)
print(f'Solution: x_mid={x_mid}, y_mid={y_mid}, n2={n2}, iters1={iters1}, iters2={iters2}, iters={iters1+iters2}')
print(f'Back calculated F={F1}, error={abs(F1-F)/F*100}%')
print(f'Diff with sym solution: diff_x={abs(x_mid-x_root)}, diff_n2={abs(n2-n2_root)}')

In [ ]:
# Calculate manually - Negative lens

x_next = 0
x_prev = 0
y_next = n2_equat_neg(x_next)
iters1 = 0
while y_next > 0:
    iters1 += 1
    x_prev = x_next
    x_next += 0.1
    y_next = n2_equat_neg(x_next)
#print(f'x_prev={x_prev}, x_next={x_next}, y_next={y_next}')
y_mid = y_next
iters2 = 0
while abs(y_mid) > 1e-8:
    iters2 += 1
    x_mid = (x_prev + x_next) / 2
    y_mid = n2_equat_neg(x_mid)
    #print(f'x_mid={x_mid}, y_mid={y_mid}, y_next={y_next}')
    if y_next < 0 and y_mid > 0:
        x_prev = x_mid
    else:
        x_next = x_mid
        y_next = y_mid
n2 = -n0*(x_mid/L)**2
F1 = focus_neg(n0, n2, L)
print(f'Solution: x_mid={x_mid}, y_mid={y_mid}, n2={n2}, iters1={iters1}, iters2={iters2}, iters={iters1+iters2}')
print(f'Back calculated F={F1}, error={abs(F1-F_neg)/F_neg*100}%')
print(f'Diff with sym solution: diff_x={abs(x_mid-x_root_neg)}, diff_n2={abs(n2-n2_root_neg)}')

In [ ]:
# Plot the shape of IOR

n2_this = 6.8
x_r = np.linspace(start=-1, stop=1, num=100)
y_n = n0 - (n2_this*x_r**2)/2
y_n_neg = n0 - (-n2_this*x_r**2)/2
fig, ax = plt.subplots(figsize=(6,4))
ax.set_xlabel('Radial distance, m')
ax.set_ylabel('IOR')
ax.grid(True, linestyle=':')
ax.plot(x_r, y_n, label='$ n_2 > 0 $')
ax.plot(x_r, y_n_neg, label='$ n_2 < 0 $')
ax.legend()

In [ ]:
# Focal range as a function of n0

x_n0 = np.linspace(1, 2.5, 100)
fig, ax = plt.subplots(figsize=(6,4))
ax.set_xlabel('$n_0$')
ax.set_ylabel('Focal range, m')
ax.grid(True, linestyle=':')
ax.plot(x_n0, focus(x_n0, 6.8, L), label='$ n_2 > 0 $')
ax.plot(x_n0, -focus_neg(x_n0, -6.8, L), label='$ n_2 > 0 $ (inverted)')
ax.legend()

In [ ]:
# Focal range as a function of rod length

print(f'n0={n0}, n2={n2}')
x_L = np.linspace(0.01, 1, 100)
fig, ax = plt.subplots(figsize=(6,4))
ax.set_xlabel('Rod length, m')
ax.set_ylabel('Focal range, m')
ax.grid(True, linestyle=':')
ax.plot(x_L, focus(n0, 6.8, x_L), label='$ n_2 > 0 $')
ax.plot(x_L, -focus_neg(n0, -6.8, x_L), label='$ n_2 < 0 $ (inverted)')
ax.legend()